In [1]:
import pandas as pd
import random

In [2]:
df = pd.read_csv("coco_cola_sales.csv")

In [3]:
def explore(df):
    with pd.option_context("display.max_colwidth", 20):
        info = pd.DataFrame()
        info['data type'] = df.dtypes
        info['percent missing'] = df.isnull().sum()*100/len(df)
        info['No. unique'] = df.apply(lambda x: len(x.unique()))
        info['unique values'] = df.apply(lambda x: x.unique())
        return info.sort_values('data type')

In [4]:
# Remove the '$' and convert the column to float
df['Price per Unit'] = df['Price per \nUnit'].str.replace('$', '', regex=False).astype(float)

In [5]:
# Remove commas and dollar signs, then convert to float
df['Operating Profit'] = df['Operating Profit'].str.replace(',', '').str.replace('$', '', regex=False).astype(float)
df['Total Sales'] = df['Total Sales'].str.replace(',', '').str.replace('$', '', regex=False).astype(float)

In [6]:
# Remove commas and convert the column to int
df['Units Sold'] = df['Units Sold'].str.replace(',', '').astype(int)

In [7]:
# Convert 'Invoice Date' to datetime format
df['Invoice Date'] = pd.to_datetime(df['Invoice Date'], format='%Y-%m-%d')

In [8]:
# Remove the '%' and convert the column to float
df['Operating Margin (%)'] = df['Operating Margin'].str.replace('%', '', regex=False).astype(float)

In [9]:
# replacing store name according to canada region
df['Retailer'] = df['Retailer'].replace({'CVS': 'FreshCo', 'Target': 'Real Canadian Superstore'})

In [10]:
df = df.drop(['Price per \nUnit', 'Operating Margin'], axis=1)

In [11]:
df = df.rename(columns={'Days to \nDeliver': 'Days to Deliver', 'Delivery \nCompany': 'Delivery Company', 
                        'Beverage Brand' : 'Beverages'})

In [12]:
explore(df)

,data type,percent missing,No. unique,unique values
Retailer ID,int64,0.0,4,"[1185732, 1197831, 1128299, 1189833]"
Invoice Date,datetime64[ns],0.0,270,"[2022-01-14 00:00:00, 2022-02-12 00:00:00, 202..."
Units Sold,int32,0.0,86,"[12000, 10000, 8500, 9000, 12500, 9500, 8250, ..."
Days to Deliver,int64,0.0,46,"[25, 34, 48, 20, 26, 43, 19, 41, 42, 37, 59, 2..."
Total Sales,float64,0.0,355,"[6000.0, 5000.0, 4000.0, 3825.0, 5400.0, 6250...."
Operating Profit,float64,0.0,691,"[3000.0, 1500.0, 1400.0, 1339.0, 1620.0, 1250...."
Price per Unit,float64,0.0,21,"[0.5, 0.4, 0.45, 0.6, 0.55, 0.65, 0.7, 0.25, 0..."
Operating Margin (%),float64,0.0,12,"[50.0, 30.0, 35.0, 25.0, 45.0, 20.0, 15.0, 40...."
Retailer,object,0.0,4,"[Walmart, Costco, FreshCo, Real Canadian Super..."
Region,object,0.0,5,"[Northeast, South, West, Midwest, Southeast]"


### Transforming Data from USA to Canada

In [13]:
# Create a sample DataFrame
data = {
    'existing_column': range(3744)  # Example existing column with 3744 entries
}
df1 = pd.DataFrame(data)

# Dictionary with state names and their respective counts
state_counts = {
    "Ontario": 977,
    "Quebec": 632,
    "Manitoba": 234,
    "Saskatchewan": 184,
    "Alberta": 663,
    "British Columbia": 542,
    "Newfoundland and Labrador": 126,
    "Prince Edward Island": 82,
    "Nova Scotia": 157,
    "New Brunswick": 65,
    "Yukon": 37,
    "Northwest Territories": 27,
    "Nunavut": 18
}

# Create a list of states based on their counts
state_list = [state for state, count in state_counts.items() for _ in range(count)]

# If the DataFrame has more rows than the total counts, repeat states to match the length
if len(state_list) < len(df1):
    state_list *= (len(df1) // len(state_list)) + 1  # Repeat states as needed
state_list = state_list[:len(df1)]  # Trim to match the DataFrame length

# Add the new column 'State1' to the DataFrame
df1['State1'] = state_list

# Define region mapping for each state
region_mapping = {
    "Newfoundland and Labrador": "Atlantic",
    "Prince Edward Island": "Atlantic",
    "Nova Scotia": "Atlantic",
    "New Brunswick": "Atlantic",
    "Ontario": "Central",
    "Quebec": "Central",
    "Manitoba": "Prairies",
    "Saskatchewan": "Prairies",
    "Alberta": "Prairies",
    "British Columbia": "West Coast",
    "Yukon": "North",
    "Northwest Territories": "North",
    "Nunavut": "North"
}

# Create the "Region1" column based on the "State1" column
df1['Region1'] = df1['State1'].map(region_mapping)

# Define mapping of states to popular cities
city_mapping = {
    "Newfoundland and Labrador": ["St. John's", "Mount Pearl", "Corner Brook", "Gander", "Labrador City"],
    "Prince Edward Island": ["Charlottetown", "Summerside", "Montague", "Stratford", "Cornwall"],
    "Nova Scotia": ["Halifax", "Sydney", "Dartmouth", "Truro", "New Glasgow"],
    "New Brunswick": ["Fredericton", "Saint John", "Moncton", "Dieppe", "Bathurst"],
    "Ontario": ["Toronto", "Ottawa", "Mississauga", "Brampton", "Hamilton", "Windsor"],
    "Quebec": ["Montreal", "Quebec City", "Laval", "Gatineau", "Sherbrooke"],
    "Manitoba": ["Winnipeg", "Brandon", "Thompson", "Steinbach", "Portage la Prairie"],
    "Saskatchewan": ["Regina", "Saskatoon", "Moose Jaw", "Prince Albert", "Yorkton"],
    "Alberta": ["Calgary", "Edmonton", "Red Deer", "Lethbridge", "Medicine Hat"],
    "British Columbia": ["Vancouver", "Victoria", "Burnaby", "Richmond", "Kamloops"],
    "Yukon": ["Whitehorse", "Dawson City", "Watson Lake", "Haines Junction", "Carcross"],
    "Northwest Territories": ["Yellowknife", "Inuvik", "Hay River", "Fort Smith", "Norman Wells"],
    "Nunavut": ["Iqaluit", "Rankin Inlet", "Arviat", "Baker Lake", "Cambridge Bay"]
}

# Randomly select a city for each state from the city mapping
df1['City1'] = df1['State1'].apply(lambda state: random.choice(city_mapping[state]))

# Drop the existing column
df1 = df1.drop(columns=["existing_column"])

df1

,State1,Region1,City1
0,Ontario,Central,Ottawa
1,Ontario,Central,Toronto
2,Ontario,Central,Brampton
3,Ontario,Central,Brampton
4,Ontario,Central,Ottawa
...,...,...,...
3739,Nunavut,North,Rankin Inlet
3740,Nunavut,North,Cambridge Bay
3741,Nunavut,North,Arviat
3742,Nunavut,North,Cambridge Bay


In [14]:
df.shape

(3744, 14)

In [15]:
df1.shape

(3744, 3)

## Merging two dataframes and removing unnecssary

In [16]:
final_df = pd.concat([df, df1], axis=1)

In [17]:
explore(final_df)

,data type,percent missing,No. unique,unique values
Retailer ID,int64,0.0,4,"[1185732, 1197831, 1128299, 1189833]"
Invoice Date,datetime64[ns],0.0,270,"[2022-01-14 00:00:00, 2022-02-12 00:00:00, 202..."
Units Sold,int32,0.0,86,"[12000, 10000, 8500, 9000, 12500, 9500, 8250, ..."
Days to Deliver,int64,0.0,46,"[25, 34, 48, 20, 26, 43, 19, 41, 42, 37, 59, 2..."
Operating Margin (%),float64,0.0,12,"[50.0, 30.0, 35.0, 25.0, 45.0, 20.0, 15.0, 40...."
Price per Unit,float64,0.0,21,"[0.5, 0.4, 0.45, 0.6, 0.55, 0.65, 0.7, 0.25, 0..."
Operating Profit,float64,0.0,691,"[3000.0, 1500.0, 1400.0, 1339.0, 1620.0, 1250...."
Total Sales,float64,0.0,355,"[6000.0, 5000.0, 4000.0, 3825.0, 5400.0, 6250...."
Retailer,object,0.0,4,"[Walmart, Costco, FreshCo, Real Canadian Super..."
State1,object,0.0,13,"[Ontario, Quebec, Manitoba, Saskatchewan, Albe..."


In [18]:
final_df = final_df.drop(['Region','State','City'], axis=1)

In [19]:
final_df = final_df.rename(columns={'Region1' : 'Region', 'State1' : 'State', 'City1' : 'City'})

In [20]:
explore(final_df)

,data type,percent missing,No. unique,unique values
Retailer ID,int64,0.0,4,"[1185732, 1197831, 1128299, 1189833]"
Invoice Date,datetime64[ns],0.0,270,"[2022-01-14 00:00:00, 2022-02-12 00:00:00, 202..."
Units Sold,int32,0.0,86,"[12000, 10000, 8500, 9000, 12500, 9500, 8250, ..."
Days to Deliver,int64,0.0,46,"[25, 34, 48, 20, 26, 43, 19, 41, 42, 37, 59, 2..."
Total Sales,float64,0.0,355,"[6000.0, 5000.0, 4000.0, 3825.0, 5400.0, 6250...."
Operating Profit,float64,0.0,691,"[3000.0, 1500.0, 1400.0, 1339.0, 1620.0, 1250...."
Price per Unit,float64,0.0,21,"[0.5, 0.4, 0.45, 0.6, 0.55, 0.65, 0.7, 0.25, 0..."
Operating Margin (%),float64,0.0,12,"[50.0, 30.0, 35.0, 25.0, 45.0, 20.0, 15.0, 40...."
Retailer,object,0.0,4,"[Walmart, Costco, FreshCo, Real Canadian Super..."
Beverages,object,0.0,6,"[Coca-Cola, Diet Coke, Sprite, Fanta, Powerade..."


In [21]:
final_df.to_csv('coco_cola_canada_sales.csv', index=False) 